In [1]:
# import requests
# import pandas as pd
# import matplotlib.pyplot as plt
# from env import key, client
# import datetime
# from datetime import date
# from datetime import datetime, timedelta
# from dateutil.relativedelta import *

# import re

# import json

# import climacell_api 
# import wrangle

In [2]:
# # This request is the San Antonio Botanical Garden. It returns realtime data for the identified fields.
# response = client.realtime(lat= 29.4576, lon=98.4587, 
#                            fields=['precipitation', 'temp', 'humidity', 'wind_speed', 'cloud_cover'])
# # Returns status code (Query Successful or Not)
# print(response.status_code)
# # Create df of results to visualize
# data = response.json()
# df = pd.DataFrame(data)
# df.head()

In [3]:
# #from climacell_api import ClimacellApiClient
# r = client.historical_station(lat=40, lon=50, start_time='2021-01-22T20:47:00Z', end_time='now', fields=['temp', 'wind_gust'])
# r.status_code

## Historical Climacell
`Data returned is a list of forecast data for a specific location up to 6 hours in the past.`

In [4]:
# stop = datetime.now().isoformat()
# start = (datetime.now() - timedelta(hours=6)).isoformat()

# url = "https://api.climacell.co/v3/weather/historical/climacell"

# querystring = {'lat': 40, 
#                'lon': 50,
#                "timestep":30,
#                "unit_system":"si",
#                "apikey":key, 
#                "start_time": start, 
#                "end_time": stop }

# response = requests.request("GET", url, params=querystring)
# data = response.json()
# df_clima = pd.DataFrame(data)
# df_clima.head(5)

## Historical Station
`24 hours in the past'

In [5]:
import pandas as pd
import numpy as np
import requests
import os
from env import key, client
import scipy as sp 
import datetime
from datetime import date
from datetime import datetime, timedelta
from dateutil.relativedelta import *

In [25]:
def api_get():
    stop = datetime.now().isoformat()
    start = (datetime.now() - relativedelta(days=1)).isoformat()

    url = "https://api.climacell.co/v3/weather/historical/station"

    querystring = {'lat': 30, 
               'lon': 40,
               "unit_system":"si",
               "apikey": key,
               "start_time": start, 
               "end_time": stop,
               "fields":['temp','wind_speed','precipitation','baro_pressure'],
              }

    response = requests.request("GET", url, params=querystring)
    data = response.json()
    df = pd.DataFrame(data)
    return df

def data_clean(df):
    #Modify Temp
    df['temp'] = df['temp'].astype(str)
    df['temp_degree_c']=df['temp'].str.extract(r'(\d+)')
    df.drop(columns =['temp'], inplace = True)
    #Modify Baro-Pressure
    df['baro_pressure'] = df['baro_pressure'].astype(str)
    df['baro_pressure_hPa']=df['baro_pressure'].str.extract(r'(\d+)')
    df.drop(columns =['baro_pressure'], inplace = True)
    #Modify Wind_speed
    df['wind_speed'] = df['wind_speed'].astype(str)
    df['wind_speed_m/s']=df['wind_speed'].str.extract(r'(\d+)')
    df.drop(columns =['wind_speed'], inplace = True)
    #Modify preciatation
    df['precipitation'] = df['precipitation'].astype(str)
    df['precipitation_mm/hr']=df['precipitation'].str.extract(r'(\d+)')
    df.drop(columns =['precipitation'], inplace = True)
    #Modify observation_time
    df['observation_time']= df['observation_time'].astype(str)
    df['observation_time']= df['observation_time'].str.extract(r'(\d+.\d+.\d+\d+.\d+.\d+)')
    df['observation_time']= df['observation_time'].str.replace('T',' ')
    df['precipitation_mm/hr'] = df['precipitation_mm/hr'].fillna(0)
    return df

def get_climacell_data():
    df = api_get()
    df = data_clean(df)
    df_old = pd.read_csv('climate_data.csv', index_col=0)
    merge = [df_old, df]
    df = pd.concat(merge)
    df.to_csv('climate_data.csv')
    return df


In [26]:
df = get_climacell_data()
df.head()

,observation_time,temp_degree_c,baro_pressure_hPa,wind_speed_m/s,precipitation_mm/hr
0,2021-01-26 14:00,19,1022,3,0
1,2021-01-26 15:00,17,1022,2,0
2,2021-01-26 16:00,14,1023,2,0
3,2021-01-26 17:00,13,1024,1,0
4,2021-01-26 18:00,12,1024,0,0


In [27]:
df.shape

(48, 5)

In [28]:
df.tail()

,observation_time,temp_degree_c,baro_pressure_hPa,wind_speed_m/s,precipitation_mm/hr
19,2021-01-27 09:00,19,1019,5,0
20,2021-01-27 10:00,23,1017,4,0
21,2021-01-27 11:00,25,1016,8,0
22,2021-01-27 12:00,26,1015,6,0
23,2021-01-27 13:00,26,1014,6,0


In [29]:
df.tail(25)

,observation_time,temp_degree_c,baro_pressure_hPa,wind_speed_m/s,precipitation_mm/hr
23,2021-01-27 13:00,26,1014,6,0
0,2021-01-26 14:00,19,1022,3,0
1,2021-01-26 15:00,17,1022,2,0
2,2021-01-26 16:00,14,1023,2,0
3,2021-01-26 17:00,13,1024,1,0
4,2021-01-26 18:00,12,1024,0,0
5,2021-01-26 19:00,12,1024,1,0
6,2021-01-26 20:00,11,1024,1,0
7,2021-01-26 21:00,10,1023,1,0
8,2021-01-26 22:00,9,1023,1,0


In [11]:
# stop = datetime.now().isoformat()
# start = (datetime.now() - relativedelta(days=1)).isoformat()

# url = "https://api.climacell.co/v3/weather/historical/station"

# querystring = {'lat': 30, 
#                'lon': 40,
#                "unit_system":"si",
#                "apikey":key,
#                "start_time": start, 
#                "end_time": stop,
#                "fields":['temp','wind_speed','precipitation','baro_pressure'],
#               }


# response = requests.request("GET", url, params=querystring)
# data = response.json()
# df_hist = pd.DataFrame(data)
# df_hist.head(5)
# # #df_hist.to_csv('climate_data.csv')

### Unpack Column and turn it in to individual colums in data frame

In [12]:
# df_hist.info()

In [13]:
# df_hist['temp'] = df_hist['temp'].astype(str)

In [14]:
# df_hist.info()

In [15]:
# #Modify Temp
# df_hist['temp'] = df_hist['temp'].astype(str)
# df_hist['temp_degree_c']=df_hist['temp'].str.extract(r'(\d+)')
# df_hist.drop(columns =['temp'], inplace = True)
# #Modify Baro-Pressure
# df_hist['baro_pressure'] = df_hist['baro_pressure'].astype(str)
# df_hist['baro_pressure_hPa']=df_hist['baro_pressure'].str.extract(r'(\d+)')
# df_hist.drop(columns =['baro_pressure'], inplace = True)
# #Modify Wind_speed
# df_hist['wind_speed'] = df_hist['wind_speed'].astype(str)
# df_hist['wind_speed_m/s']=df_hist['wind_speed'].str.extract(r'(\d+)')
# df_hist.drop(columns =['wind_speed'], inplace = True)
# #Modify preciatation
# df_hist['precipitation'] = df_hist['precipitation'].astype(str)
# df_hist['precipitation_mm/hr']=df_hist['precipitation'].str.extract(r'(\d+)')
# df_hist.drop(columns =['precipitation'], inplace = True)
# #Modify observation_time
# df_hist['observation_time']= df_hist['observation_time'].astype(str)
# df_hist['observation_time']= df_hist['observation_time'].str.extract(r'(\d+.\d+.\d+\d+.\d+.\d+)')
# df_hist['observation_time']= df_hist['observation_time'].str.replace('T',' ')
# df_hist['precipitation_mm/hr'] = df_hist['precipitation_mm/hr'].fillna(0)
# df_hist.head()

In [16]:
# df_hist.info()

#### Dictionary to Unpack

In [17]:
# temp.drop(columns=['units'], inplace = True)
# temp_df = pd.DataFrame(temp)
# temp_df
# tf = pd.DataFrame([
#         {'id': 1, 'nested': {'a': 1, 'b': 2} },
#         {'id': 2, 'nested': {'a': 2, 'b': 4} },
#         {'id': 3, 'nested': {'a': 3, 'b': 6} },
#         {'id': 4, 'nested': {'a': 4}},
#     ])
# tf.head()

#### Function to Unpack and Create Columns

In [18]:
# def unpack(df, column, fillna=None):
#     ret = None
#     if fillna is None:
#         ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems()))], axis=1)
#         del ret[column]
#     else:
#         ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems())).fillna(fillna)], axis=1)
#         del ret[column]
#     return ret

# unpack(tf, 'nested', 0)

In [19]:
# df = pd.concat([df, pd.DataFrame(list(df['run_info'])).fillna(0).astype(int)], axis=1)

### Append to DF

In [20]:
# result = pd.concat([df_hist, temp], axis=1)
# result

In [21]:
# now = datetime.now()
# year = '{:02d}'.format(now.year)
# month = '{:02d}'.format(now.month)
# day = '{:02d}'.format(now.day)
# hour = '{:02d}'.format(now.hour)
# minute = '{:02d}'.format(now.minute)
# day_month_year_hour_now = '{}-{}-{}-{}'.format(year, month, day, hour)
# day_month_year_hour_now

In [22]:
# six_hour_ago = datetime.now() - timedelta(hours=6)
# year = '{:02d}'.format(six_hour_ago.year)
# month = '{:02d}'.format(six_hour_ago.month)
# day = '{:02d}'.format(six_hour_ago.day)
# hour = '{:02d}'.format(six_hour_ago.hour)
# minute = '{:02d}'.format(six_hour_ago.minute)
# day_month_year_hour_past = '{}-{}-{}-{}'.format(year, month, day, hour)
# day_month_year_hour_past

In [23]:
# start = datetime.now().isoformat()
# start

In [24]:
# stop = (datetime.now() - relativedelta(months=4)).isoformat()
# stop